In [1]:
import pandas as pd
import json
import os
import nest_asyncio
from typing import List, Optional, Literal
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig, CacheMode
from pydantic import BaseModel, Field
from google import genai
from google.genai import types
from dotenv import load_dotenv
import re

### Filter List

In [2]:
df = pd.read_json('venues.json')

In [3]:
df.head()

,id,name,location,contact,type,tagline,price_range_id,rating,num_ratings,url_slug,enable_discovery,enable_for_amex,deep_link,metadata,web_link
0,"{'resy': 84342, 'foursquare': None, 'google': ...",Like Music VIP Cancún,{'address_1': 'Av Xcaret Supermanzana 35 Manza...,"{'phone_number': None, 'url': None}",Cocktail Bar,None,2,NaN,0.0,like-music-vip-cancun,1,1,resy://resy.com/VenueDetails?venue_id=%7B%27re...,{'description': ' Like Music VIP Cancún is a C...,"https://resy.com/?venue_id={'resy': 84342, 'fo..."
1,"{'resy': 81110, 'foursquare': None, 'google': ...",Restaurante Da Enzo Playa del Carmen,"{'address_1': 'Calle 42 Entre Av. 10 y, 5 Av. ...","{'phone_number': None, 'url': None}",Italian,None,2,NaN,0.0,restaurante-da-enzo-playa-del-carmen,1,1,resy://resy.com/VenueDetails?venue_id=%7B%27re...,{'description': ' Restaurante Da Enzo Playa de...,"https://resy.com/?venue_id={'resy': 81110, 'fo..."
2,"{'resy': 78530, 'foursquare': None, 'google': ...",Restaurante El Plebe Bichi Teotihuacan,"{'address_1': 'Calle Emilio Carranza 222, 5585...","{'phone_number': None, 'url': None}",Seafood,None,2,NaN,0.0,restaurante-el-plebe-bichi-teotihuacan,1,1,resy://resy.com/VenueDetails?venue_id=%7B%27re...,{'description': ' Restaurante El Plebe Bichi T...,"https://resy.com/?venue_id={'resy': 78530, 'fo..."
3,"{'resy': 78730, 'foursquare': None, 'google': ...",Restaurante La Mentirosa Los Mochis,"{'address_1': 'Blvd Centenario 805, Centro, 81...","{'phone_number': None, 'url': None}",International,None,2,NaN,0.0,restaurante-la-mentirosa-los-mochis,1,1,resy://resy.com/VenueDetails?venue_id=%7B%27re...,{'description': ' Restaurante La Mentirosa Los...,"https://resy.com/?venue_id={'resy': 78730, 'fo..."
4,"{'resy': 75788, 'foursquare': None, 'google': ...",Restaurante Salmone's Morelia Suc. Siervo,"{'address_1': 'Av Siervo de La Nacion s/n, Agu...","{'phone_number': None, 'url': None}",Seafood,None,2,NaN,0.0,restaurante-salmones-morelia-suc-siervo,1,1,resy://resy.com/VenueDetails?venue_id=%7B%27re...,{'description': ' Restaurante Salmone's Moreli...,"https://resy.com/?venue_id={'resy': 75788, 'fo..."


In [4]:
df_flat = pd.json_normalize(df['location'])

In [5]:
df = pd.concat([df, df_flat.add_prefix('loc_')], axis=1)

In [6]:
df_nyc = df[df["loc_url_slug"] == 'new-york-ny'].reset_index(drop=True)

In [7]:
df_nyc_flat_url = pd.json_normalize(df_nyc['contact'])
df_nyc_flat_id = pd.json_normalize(df_nyc['id'])

In [8]:
df_nyc = df_nyc.drop(columns=['contact']).join(df_nyc_flat_url)
df_nyc = df_nyc.drop(columns=['id']).join(df_nyc_flat_id)

In [9]:
df_nyc = df_nyc.drop(columns=['location'])

In [10]:
df_nyc.columns

Index(['name', 'type', 'tagline', 'price_range_id', 'rating', 'num_ratings',
       'url_slug', 'enable_discovery', 'enable_for_amex', 'deep_link',
       'metadata', 'web_link', 'loc_address_1', 'loc_address_2',
       'loc_locality', 'loc_region', 'loc_postal_code', 'loc_cross_street_1',
       'loc_cross_street_2', 'loc_longitude', 'loc_latitude',
       'loc_neighborhood', 'loc_time_zone', 'loc_url_slug', 'loc_id',
       'phone_number', 'url', 'resy', 'foursquare', 'google'],
      dtype='object')

In [11]:
enrichdf = df_nyc[["resy", "foursquare", "google", "name", "type", "price_range_id", "rating", "num_ratings", "web_link", "loc_id", "loc_neighborhood", "phone_number", "url"]]

In [12]:
enrichdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2467 entries, 0 to 2466
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   resy              2467 non-null   int64  
 1   foursquare        1391 non-null   object 
 2   google            2466 non-null   object 
 3   name              2467 non-null   object 
 4   type              2467 non-null   object 
 5   price_range_id    2467 non-null   int64  
 6   rating            2439 non-null   float64
 7   num_ratings       2439 non-null   float64
 8   web_link          2467 non-null   object 
 9   loc_id            2467 non-null   object 
 10  loc_neighborhood  2467 non-null   object 
 11  phone_number      2293 non-null   object 
 12  url               2420 non-null   object 
dtypes: float64(2), int64(2), object(9)
memory usage: 250.7+ KB


### Crawl4All 

In [13]:
load_dotenv()

True

In [14]:
nest_asyncio.apply()

In [15]:
client = genai.Client(api_key=os.getenv("GENAI_API_KEY")) 

#### Define Data Models

In [24]:
class ClassifiedLink(BaseModel):
    url: str = Field(description="The href URL from the input.")
    text: str = Field(description="The anchor text from the input. May be gibberish.")
    category: Literal['ordering', 'gift_card', 'instagram', 'private_events', 'other'] = Field(
        description="The category: 'ordering', 'gift_card', 'instagram', 'private_events', or 'other'."
    )

class LinkCollection(BaseModel):
    classified_links: List[ClassifiedLink]

class ClassificationBatch(BaseModel):
    categories: List[Literal['ordering', 'gift_card', 'instagram', 'private_events', 'other']]

class RestaurantTechProfile(BaseModel):
    pos_system: Optional[str] = Field(None, description="Inferred POS (Toast, Square, etc)")
    tech_stack: List[str] = Field(default_factory=list, description="Other systems (Bentobox, OpenTable, etc)")
    ordering_provider: Optional[str] = Field(None, description="Who powers the online ordering?")
    instagram_handle: Optional[str] = Field(None, description="The extracted handle (e.g. 'thesmithnyc')")
    tripleseat_status: str = Field("Not Found", description="Confirmed, Suspected, or Not Found")


#### Categorization LLM Helper (GeminiFlash)

In [ ]:
def classify_links_flash(links: List[dict]) -> LinkCollection:
    """
    Uses gemini-flash-latest (Gemini 2.5) to categorize links 
    instead of guessing with keywords.
    """
    # Prepare batch for LLM
    candidates = links[:100]
    
    #print("candidates links count:", len(candidates))
    
    if not candidates:
        return LinkCollection(classified_links=[])

    # We ask for a Map of Index -> Category to keep response small
    prompt = f"""
    You are a restaurant bot link classifier.
    Classify the following links based on their text and href into:
    - 'ordering' (online ordering, takeout, delivery, 'order now', point of sale, POS)
    - 'gift_card' (Gift cards, merch, store)
    - 'instagram' (Social media links to Instagram)
    - 'private_events' (Private dining, event booking, party reservations)
    - 'other' (Menus, about, contact, locations, reservations)

    If the text is gibberish or empty, RELY MORE on the href URL to classify.

    Input Links:
    {json.dumps(candidates)}
    """

    try:
        response = client.models.generate_content(
            model="gemini-flash-latest", # CURRENTLY GEMINI 2.5 FLASH
            contents=prompt,
            config={
                "response_mime_type": "application/json",
                "response_json_schema": ClassificationBatch.model_json_schema(),
            }
        )
        
        # Parse map and rebuild list
        batch_result = ClassificationBatch.model_validate_json(response.text)
        final_links = []
        for i, link in enumerate(candidates):
            category = "other" # Default fallback
            if i < len(batch_result.categories):
                category = batch_result.categories[i]
                
            final_links.append(ClassifiedLink(
                url=link.get("href", "") or link.get("url", ""),
                text=link.get("text", ""),
                category=category
            ))

        return LinkCollection(classified_links=final_links)

    except Exception as e:
        print(f"Flash Classification Error: {e}")
        return LinkCollection(classified_links=[])

#### Analyze Tech Stack (Gemini3)

In [ ]:
def analyze_tech_stack_gemini3(
    classified_links: List[ClassifiedLink], 
    script_domains: List[str], 
    footer_text: str,
    deep_dive_signals: List[str]
) -> RestaurantTechProfile:
    """
    Uses gemini-3-pro-preview to reason about the signals found.
    """
    
    # Organize data for the model
    ordering_urls = [l.url for l in classified_links if l.category == "ordering"]
    gift_urls = [l.url for l in classified_links if l.category == "gift_card"]
    socials = [l.url for l in classified_links if l.category == "instagram"]
    
    prompt = f"""
    Analyze these signals to determine the Restaurant's Tech Stack.
    
    1. Validated Ordering Links: {ordering_urls}
    2. Validated Gift Card Links: {gift_urls}

    3. **Deep Dive Signals (Ordering/Gift Pages):** 
    {json.dumps(deep_dive_signals, indent=2)}
    (IMPORTANT: These are links/redirects found AFTER clicking the ordering/gift buttons. 
     Look here for 3rd party POS domains like 'toasttab.com', 'spoton.com', 'clover.com'.)

    4. Loaded Scripts/Domains: {script_domains}
    5. Footer Text: {footer_text}
    6. Social Links: {socials}
    
    Task:
    - Identify the POS System (Point of Sale). 
      - PRIORITY: Look at "Deep Dive Signals". If a link redirects to or points to a known POS (Toast, Square, SpotOn, Upserve, Aloha, Heartland, Clover etc), that is the POS.
      - SECONDARY: Look at Scripts.
    - Identify the Website Builder (e.g., Bentobox, Squarespace).
    - Extract the Instagram Handle.
    """

    #print("gemini3", prompt)

    try:
        response = client.models.generate_content(
            model="gemini-3-pro-preview", # DEEP REASONING MODEL
            contents=prompt,
            config={
                "thinking_config": types.ThinkingConfig(thinking_level="low"),
                "response_mime_type": "application/json",
                "response_json_schema": RestaurantTechProfile.model_json_schema(),
            }
        )

        techprofile = RestaurantTechProfile.model_validate_json(response.text)
        print("gemini3profile", techprofile)
        return techprofile
    except Exception as e:
        print(f"Gemini 3 Error: {e}")
        return RestaurantTechProfile()

#### Other Helper Functions

In [27]:
def extract_footer(soup):
    # --- LEVEL 1: The "Smoking Gun" Selectors (Specific Builders) ---
    # These attributes are definitive proof that an element is a footer template.
    high_priority_selectors = [
        '[data-elementor-post-type="footer"]',  # <--- THIS FIXES YOUR SPECIFIC HTML
        '[data-elementor-type="footer"]',       # Standard Elementor
        '.elementor-location-footer',           # Elementor Theme Builder
        '#footer-builder',
        '.fusion-footer',                       # Avada
        '.divi-builder #main-footer',           # Divi
    ]

    for selector in high_priority_selectors:
        element = soup.select_one(selector)
        if element:
            print(f"Match found via High Priority: {selector}")
            return clean_text(element)

    # --- LEVEL 2: The "Name Game" (Class/ID Search) ---
    # We look for classes/IDs containing "footer", but we filter them.
    # We don't want to accidentally grab a wrapper like <div id="page-wrapper-footer-included">
    candidates = []
    
    # Find ALL divs, sections, and footers
    tags = soup.find_all(['div', 'section', 'footer'])
    
    for tag in tags:
        # Get string of classes and ID
        classes = " ".join(tag.get('class', []))
        id_val = tag.get('id', "")
        identifier = (classes + " " + id_val).lower()
        
        # Check if it identifies as a footer
        if 'footer' in identifier or 'colophon' in identifier:
            # FILTER: Ignore elements that are too large (likely page wrappers)
            # A footer usually has fewer than 2000 characters of text.
            text_len = len(tag.get_text(strip=True))
            if 10 < text_len < 2500:
                candidates.append(tag)

    # If we found candidates, return the LAST one in the DOM (closest to bottom)
    if candidates:
        print("Match found via Class/ID Search")
        return clean_text(candidates[-1])

    # --- LEVEL 3: Text Anchors (Copyright & Developer Credits) ---
    # Your example didn't have "Copyright", but it had "Site Made With Love".
    # We look for these phrases and grab their parent container.
    keywords = [
        r'©', 
        r'&copy;', 
        r'copyright', 
        r'all rights reserved', 
        r'powered by', 
        r'made with love', 
        r'made by', 
        r'designed by'
    ]
    
    pattern = re.compile('|'.join(keywords), re.IGNORECASE)
    
    # Find text nodes matching keywords
    matches = soup.find_all(string=pattern)
    
    if matches:
        # Get the last match (closest to bottom)
        target = matches[-1]
        
        # Walk up 3 levels to find a container (div or section)
        parent = target.parent
        for _ in range(3):
            if parent.name in ['div', 'section', 'footer', 'aside']:
                print(f"Match found via Text Anchor: {target.strip()[:20]}...")
                return clean_text(parent)
            parent = parent.parent

    return ""

def clean_text(element):
    """Clean up whitespace but preserve readability"""
    # Get text with newline separators
    text = element.get_text("\n", strip=True)
    # Remove excessive newlines
    text = re.sub(r'\n\s*\n', '\n', text)
    return text

#### Main Logic

In [28]:
async def process_restaurant(crawler, start_url, config):
    print(f"--- Processing: {start_url} ---")
    
    # 1. Crawl Homepage
    result = await crawler.arun(url=start_url, config=config)
    if not result.success: return None

    # 2. Extract Basic Signals (Scripts & Footer)
    soup = BeautifulSoup(result.html, 'html.parser')
    scripts = set()
    for s in soup.find_all('script', src=True):
        domain = urlparse(s.get('src')).netloc
        if domain: scripts.add(domain)
    
    footer = extract_footer(soup)
    
    # Check Tripleseat on Homepage
    ts_found = False
    if "tripleseat.com" in result.html or soup.find(id="tripleseat-form"):
        ts_found = True

    # 3. USE GEMINI FLASH: Classify Links
    # We merge internal and external links for classification
    all_links = [{"text": l['text'], "href": urljoin(start_url, l['href'])} 
                 for l in result.links.get('internal', []) + result.links.get('external', [])]
    
    link_collection = classify_links_flash(all_links)
    classified_links = link_collection.classified_links

    # We visit these pages to find:
    # A) Redirects (e.g. /order -> toasttab.com)
    # B) Links ON that page (e.g. /order -> Button href="toasttab.com")
    deep_dive_signals = []
    ordering_candidates = [l for l in classified_links if l.category == "ordering"]
    gift_candidates = [l for l in classified_links if l.category == "gift_card"]
    urls_to_drill = (ordering_candidates + gift_candidates)[:4]

    for link_obj in urls_to_drill:
        print(f"  > Drilling down into tech link: {link_obj.url}")
        try:
            sub_res = await crawler.arun(url=link_obj.url)

            if sub_res.success:
                # Signal A: Did we get redirected?
                # Compare the final URL to the one we clicked.
                # If we clicked /order and ended up on toasttab.com, that's a strong signal.
                if urlparse(sub_res.url).netloc != urlparse(link_obj.url).netloc:
                     deep_dive_signals.append(f"Redirect from {link_obj.text}: {sub_res.url}")

                # Signal B: Scan for External Links on this sub-page
                # This handles the case where the page is internal but contains a button to the POS.
                # We extract external links found on this sub-page.
                external_links = sub_res.links.get("external", [])

                for ext_link in external_links[:10]:
                    href = ext_link.get('href', '')
                    if href:
                        deep_dive_signals.append(f"Link on '{link_obj.text}' page: {href}")

                # Signal C: Capture scripts on this sub-page
                sub_soup = BeautifulSoup(sub_res.html, 'html.parser')
                for s in sub_soup.find_all('script', src=True):
                    domain = urlparse(s.get('src')).netloc
                    if domain: scripts.add(domain)
        except Exception as e:
            print(f"  ! Failed to drill down {link_obj.url}: {e}")
    
    # 4. USE GEMINI 3: Analyze Tech Stack
    # We pass the *clean, categorized* data to the smart model
    tech_profile = analyze_tech_stack_gemini3(
        classified_links, 
        list(scripts)[:50], 
        footer,
        deep_dive_signals
    )

    if ts_found: 
        tech_profile.tripleseat_status = "Confirmed (Homepage)"

    # 5. Navigate to Private Events (if not already found)
    # We look for the link categorized as 'private_events' by Flash
    events_link = next((l for l in classified_links if l.category == "private_events"), None)
    if events_link:
        print(f"  > Flash identified Events page: {events_link.url}")
        try:
            evt_res = await crawler.arun(url=events_link.url)
            if evt_res.success:
                if "tripleseat.com" in evt_res.html or "tripleseat" in evt_res.html.lower():
                    tech_profile.tripleseat_status = "Confirmed (Events Page)"
                elif tech_profile.tripleseat_status == "Not Found":
                    tech_profile.tripleseat_status = "Not Found on Events Page"
        except Exception as e:
            print(f"  ! Failed to process Events page {events_link.url}: {e}")

    return {
        "url": start_url,
        "pos": tech_profile.pos_system,
        "stack": tech_profile.tech_stack,
        "instagram": tech_profile.instagram_handle,
        "tripleseat": tech_profile.tripleseat_status,
        "ordering_url": next((l.url for l in classified_links if l.category == "ordering"), None)
    }
    

In [29]:
async def main():
    
    urls = ["https://www.andsonnyc.com"]
    
    results = []
    crawler_run_config = CrawlerRunConfig(cache_mode=CacheMode.BYPASS)
    async with AsyncWebCrawler(verbose=False) as crawler:
        for url in urls:
            try:
                data = await process_restaurant(crawler, url, config=crawler_run_config)
                if data: results.append(data)
            except Exception as e:
                print(f"Error on {url}: {e}")

    df = pd.DataFrame(results)

    final_df = pd.merge(df, enrichdf, on='url', how='left')

    display(final_df)

In [30]:
await main()

[INIT].... → Crawl4AI 0.7.7 

--- Processing: https://www.andsonnyc.com ---


[FETCH]... ↓ https://www.andsonnyc.com                                                                            |
✓ | ⏱: 1.23s 

[SCRAPE].. ◆ https://www.andsonnyc.com                                                                            |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.andsonnyc.com                                                                            |
✓ | ⏱: 1.24s 

Match found via Class/ID Search
candidates links count: 7
gemini3 
    Analyze these signals to determine the Restaurant's Tech Stack.

    1. Validated Ordering Links: []
    2. Validated Gift Card Links: []

    3. **Deep Dive Signals (Ordering/Gift Pages):** 
    []
    (IMPORTANT: These are links/redirects found AFTER clicking the ordering/gift buttons. 
     Look here for 3rd party POS domains like 'toasttab.com', 'spoton.com', 'clover.com'.)

    4. Loaded Scripts/Domains: ['www.googletagmanager.com', 'cdn.jsdelivr.net', 'cdn.lightwidget.com', 'unpkg.com', 'www.gstatic.com', 'ajax.googleapis.com', 'www.google.com']
    5. Footer Text: Greenwich Village
62 West 9th Street
between 5th & 6th, NYC
212.933.1193
KITCHEN HOURS
Sunday - Wednesday: 5pm - 10pm
Thursday - Saturday: 5pm - 11pm
Lunch: Saturday – Sunday 11:30am - 4pm
Bar Open Late
Join Our Mailing List
First Name:
Last Name:
E-mail:
Submit
CopyRight © 2023 All Rights Reserved. | Site by:
STUDIO
ALITY
    6. Social Links: ['htt

,url,pos,stack,instagram,tripleseat,ordering_url,resy,foursquare,google,name,type,price_range_id,rating,num_ratings,web_link,loc_id,loc_neighborhood,phone_number
0,https://www.andsonnyc.com,None,[LightWidget],andsonnyc,Not Found,None,65574,null,ChIJr77gGUhZwokRLB-4dkbWkGY,& Son Greenwich Village,Steakhouse,3,4.48097,2890.0,"https://resy.com/?venue_id={'resy': 65574, 'fo...",ny,Greenwich Village,+12129331193


#### Tester

In [24]:
urls = [
        "https://www.andsonnyc.com",
        "https://www.rezdora.nyc/"
    ]
    
results = []
crawler_run_config = CrawlerRunConfig(cache_mode=CacheMode.BYPASS)
async with AsyncWebCrawler(verbose=False) as crawler:
    for url in urls:
        try:
            data = await process_restaurant(crawler, url, config=crawler_run_config)
            if data: results.append(data)
        except Exception as e:
            print(f"Error on {url}: {e}")

df = pd.DataFrame(results)
display(df)

[INIT].... → Crawl4AI 0.7.7 

--- Processing: https://www.andsonnyc.com ---


[FETCH]... ↓ https://www.andsonnyc.com                                                                            |
✓ | ⏱: 1.15s 

[SCRAPE].. ◆ https://www.andsonnyc.com                                                                            |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.andsonnyc.com                                                                            |
✓ | ⏱: 1.16s 

--- Processing: https://www.rezdora.nyc/ ---


[FETCH]... ↓ https://www.rezdora.nyc/                                                                             |
✓ | ⏱: 1.24s 

[SCRAPE].. ◆ https://www.rezdora.nyc/                                                                             |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.rezdora.nyc/                                                                             |
✓ | ⏱: 1.25s 

,result,soup,scripts,footer,ts_found
0,(url='https://www.andsonnyc.com' html='<!DOCTY...,"[html, [[<meta content=""A7vZI3v+Gz7JfuRolKNM4A...","{www.googletagmanager.com, cdn.jsdelivr.net, u...",Greenwich Village 62 West 9th Street between 5...,False
1,(url='https://www.rezdora.nyc/' html='<!DOCTYP...,"[html, [[<meta content=""A7vZI3v+Gz7JfuRolKNM4A...","{app-assets.getbento.com, theme-assets.getbent...",Facebook Twitter Instagram Hours & Location Me...,False


In [25]:
all_links = [{"text": l['text'], "href": urljoin("https://www.rezdora.nyc/", l['href'])} 
                 for l in df["result"][1].links.get('internal', []) + df["result"][1].links.get('external', [])]

In [26]:
classify_test_results = classify_links_flash(all_links)

In [27]:
test_links = classify_test_results.classified_links

In [33]:
deep_dive_signals = []
scripts = set()
ordering_candidates = [l for l in test_links if l.category == "ordering"]
gift_candidates = [l for l in test_links if l.category == "gift_card"]
urls_to_drill = (ordering_candidates + gift_candidates)[:4]

In [37]:
for link_obj in urls_to_drill:
        print(f"  > Drilling down into tech link: {link_obj.url}")
        try:
            async with AsyncWebCrawler(verbose=False) as crawler:
                sub_res = await crawler.arun(url=link_obj.url, config=crawler_run_config)

                if sub_res.success:
                    # Signal A: Did we get redirected?
                    # Compare the final URL to the one we clicked.
                    # If we clicked /order and ended up on toasttab.com, that's a strong signal.
                    if urlparse(sub_res.url).netloc != urlparse(link_obj.url).netloc:
                        deep_dive_signals.append(f"Redirect from {link_obj.text}: {sub_res.url}")

                    # Signal B: Scan for External Links on this sub-page
                    # This handles the case where the page is internal but contains a button to the POS.
                    # We extract external links found on this sub-page.
                    external_links = sub_res.links.get("external", [])

                    for ext_link in external_links[:15]:
                        href = ext_link.get('href', '')
                        if href:
                            deep_dive_signals.append(f"Link on '{link_obj.text}' page: {href}")

                    # Signal C: Capture scripts on this sub-page
                    sub_soup = BeautifulSoup(sub_res.html, 'html.parser')
                    for s in sub_soup.find_all('script', src=True):
                        domain = urlparse(s.get('src')).netloc
                        if domain: scripts.add(domain)
        except Exception as e:
            print(f"  ! Failed to drill down {link_obj.url}: {e}")

  > Drilling down into tech link: https://www.rezdora.nyc/gift-cards


[INIT].... → Crawl4AI 0.7.7 

[FETCH]... ↓ https://www.rezdora.nyc/gift-cards                                                                   |
✓ | ⏱: 1.11s 

[SCRAPE].. ◆ https://www.rezdora.nyc/gift-cards                                                                   |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.rezdora.nyc/gift-cards                                                                   |
✓ | ⏱: 1.12s 

In [41]:
tech_profile = analyze_tech_stack_gemini3(
        test_links, 
        list(scripts)[:50], 
        results[1]['footer'],
        deep_dive_signals # <--- Passing the deep dive content
    )

In [43]:
events_link = next((l for l in test_links if l.category == "private_events"), None)
if events_link:
    print(f"  > Flash identified Events page: {events_link.url}")
    try:
        async with AsyncWebCrawler(verbose=False) as crawler:
            evt_res = await crawler.arun(url=events_link.url)
            if evt_res.success:
                if "tripleseat.com" in evt_res.html or "tripleseat" in evt_res.html.lower():
                    tech_profile.tripleseat_status = "Confirmed (Events Page)"
                elif tech_profile.tripleseat_status == "Not Found":
                    tech_profile.tripleseat_status = "Not Found on Events Page"
    except Exception as e:
            print(f"  ! Failed to drill down {events_link.url}: {e}")

  > Flash identified Events page: https://www.rezdora.nyc/private-events


[INIT].... → Crawl4AI 0.7.7 

[FETCH]... ↓ https://www.rezdora.nyc/private-events                                                               |
✓ | ⏱: 1.95s 

[SCRAPE].. ◆ https://www.rezdora.nyc/private-events                                                               |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.rezdora.nyc/private-events                                                               |
✓ | ⏱: 1.96s 

In [44]:
tech_profile

RestaurantTechProfile(pos_system='Toast', tech_stack=['BentoBox', 'Resy', 'AudioEye', 'Google Maps'], ordering_provider=None, instagram_handle='rezdoranyc', tripleseat_status='Confirmed (Events Page)')